# SBTi Temperature Alignment tool examples
This notebook provides some examples on how to use the SBTi Temperature Alignment tool.

## Install the SBTi Python module
This is only required if you didn't already instally the module.

In [ ]:
!pip install SBTi

In [1]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

## Download the dummy data provider

In [2]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

#### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [3]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

### Create a data provider
Data providers let you connect to the data source of your choice.

In [4]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

### Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

In [5]:
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")

In [6]:
portfolio.head(5)

,company_name,company_id,ISIN,weights,investment_value,engagement_target
0,"Advanced Micro Devices, Inc",US0079031078,US0079031078,6.36%,35000000,1.0
1,Adobe Systems Inc.,US00724F1012,US00724F1012,1.82%,10000000,NaN
2,Capgemini Group,FR0000125338,FR0000125338,1.82%,10000000,NaN
3,"Cisco Systems, Inc.",US17275R1023,US17275R1023,1.82%,10000000,NaN
4,Coca-Cola HBC AG,CH0198251305,CH0198251305,1.82%,10000000,NaN


To load the data from the data provider, we have to pass a list of dictionaries, containing a "company_name" and "company_id".

In [7]:
portfolio["company_isin"] = portfolio.ISIN
companies = [SBTi.interfaces.PortfolioCompany.parse_obj(company) for company in portfolio.to_dict(orient="records")]
print(portfolio.to_dict(orient="records"))

[{'company_name': 'Advanced Micro Devices, Inc', 'company_id': 'US0079031078', 'ISIN': 'US0079031078', 'weights': '6.36%', 'investment_value': 35000000, 'engagement_target': 1.0, 'company_isin': 'US0079031078'}, {'company_name': 'Adobe Systems Inc.', 'company_id': 'US00724F1012', 'ISIN': 'US00724F1012', 'weights': '1.82%', 'investment_value': 10000000, 'engagement_target': nan, 'company_isin': 'US00724F1012'}, {'company_name': 'Capgemini Group', 'company_id': 'FR0000125338', 'ISIN': 'FR0000125338', 'weights': '1.82%', 'investment_value': 10000000, 'engagement_target': nan, 'company_isin': 'FR0000125338'}, {'company_name': 'Cisco Systems, Inc.', 'company_id': 'US17275R1023', 'ISIN': 'US17275R1023', 'weights': '1.82%', 'investment_value': 10000000, 'engagement_target': nan, 'company_isin': 'US17275R1023'}, {'company_name': 'Coca-Cola HBC AG', 'company_id': 'CH0198251305', 'ISIN': 'CH0198251305', 'weights': '1.82%', 'investment_value': 10000000, 'engagement_target': nan, 'company_isin': '

### Load the data
We'll call the data provider and ask it to provide us with the targets and the company data. This will take care of all required preprocessing.

In [8]:
portfolio_data = SBTi.utils.get_data([provider], companies)

### Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [9]:
temperature_score = TemperatureScore(
    time_frames=list(SBTi.interfaces.ETimeFrames), 
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],
    aggregation_method=PortfolioAggregationMethod.WATS
)
amended_portfolio = temperature_score.calculate(portfolio_data)

In [10]:
amended_portfolio.head(10)

,achieved_reduction,base_year,base_year_ghg_s1,base_year_ghg_s2,base_year_ghg_s3,company_id,coverage_s1,coverage_s2,coverage_s3,end_year,...,annual_reduction_rate,slope,samplesize,model,variable,param,intercept,r2,temperature_score,temperature_results
0,0.000,2015.0,2.202211e+05,310630.278165,NaN,US0079031078,1.00,1.00,NaN,2020.0,...,0.060000,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,1.30,0.000000
1,0.844,2014.0,2.794097e+06,544597.926996,NaN,US0079031078,NaN,NaN,0.95,2022.0,...,0.138571,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,0.00,0.000000
2,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,0.35,0.000000
3,0.740,2013.0,0.000000e+00,0.000000,NaN,US0079031078,0.86,0.86,NaN,2030.0,...,0.027785,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,1.83,0.000000
4,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.20,1.000000
5,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,2.83,0.727326
6,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
7,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
8,NaN,NaN,NaN,NaN,NaN,US0079031078,NaN,NaN,NaN,NaN,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
9,NaN,NaN,NaN,NaN,NaN,US00724F1012,NaN,NaN,NaN,NaN,...,NaN,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,3.20,1.000000


### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb"). The termperature scores are calculated per time-frame/scope combination.

In [11]:
def print_aggregations(aggregations):
    aggregations = aggregations.dict()
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

In [12]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.2349090909090905 degrees celcius
short - S3: 2.8205454545454542 degrees celcius
short - S1S2S3: 2.670272727272727 degrees celcius
mid - S1S2: 2.7351818181818177 degrees celcius
mid - S3: 3.0340000000000003 degrees celcius
mid - S1S2S3: 2.9639090909090906 degrees celcius
long - S1S2: 3.122181818181818 degrees celcius
long - S3: 3.2258181818181817 degrees celcius
long - S1S2S3: 3.199636363636363 degrees celcius


### Portfolio coverage

Gives insights in the coverage of portfolio entities that has set GHG emissions reduction targets, STBi-validated or otherwise. Companies with SBTi status Approved have a score of 100%, all others have a score of 0%. 

This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb").

In [13]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
round(coverage * 100) / 100

35.45